In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
from jax.lib import xla_bridge
print("jax backend {}".format(xla_bridge.get_backend().platform))

from SSN_classes import SSN2DTopoV1_ONOFF_local
from training import new_two_stage_training
from parameters import grid_pars, filter_pars, stimuli_pars, sig_pars, ssn_pars, conn_pars_m, conn_pars_s, ssn_layer_pars, conv_pars, training_pars, loss_pars

jax backend cpu


In [2]:
# Calculate A by using the default find_A function of SSN2DTopoV1_ONOFF_local when A is undefined
ssn_mid = SSN2DTopoV1_ONOFF_local(
    ssn_pars=ssn_pars, # got c_E and c_I as new parameters
    grid_pars=grid_pars,
    conn_pars=conn_pars_m,
    filter_pars=filter_pars,
    J_2x2 = ssn_layer_pars.J_2x2_m,
    gE = ssn_layer_pars.gE_m, 
    gI = ssn_layer_pars.gE_s
)

ssn_pars.A = ssn_mid.A
if ssn_pars.phases == 4:
    ssn_pars.A2 = ssn_mid.A2

####################### TRAINING PARAMETERS #############################
constant_ssn_pars = dict(
    ssn_pars=ssn_pars,
    grid_pars=grid_pars,
    conn_pars_m=conn_pars_m,
    conn_pars_s=conn_pars_s,
    gE=ssn_layer_pars.gE,
    gI=ssn_layer_pars.gI,
    filter_pars=filter_pars,
    conv_pars=conv_pars,# this got a lot of new stuff in it that might be a problem
    loss_pars=loss_pars,
    noise_type="poisson",
)

In [3]:
########### TRAINING ############
([ssn_layer_pars_dict, readout_pars],
    val_loss_per_epoch,
    training_losses,
    training_accs,
    train_sig_inputs,
    train_sig_outputs,
    val_sig_inputs,
    val_sig_outputs,
    epoch_c,
    save_w_sigs) = new_two_stage_training(
    ssn_layer_pars,
    sig_pars,
    training_pars,
    constant_ssn_pars,
    stimuli_pars,
    extra_stop=2,
)

New orientation map created for the middle layer.
#### NOT SAVING! ####


TypeError: _SSN_Base.fixed_point_r() got an unexpected keyword argument 'verbose'

In [5]:
from util import take_log
print(ssn_layer_pars_dict['logJ_2x2'][0]-take_log(ssn_layer_pars.J_2x2_m))
print(ssn_layer_pars_dict['logJ_2x2'][1]- take_log(ssn_layer_pars.J_2x2_s))

[[-0.0085609   0.00859194]
 [ 0.00856054 -0.00857073]]
[[-0.00707686  0.00956553]
 [ 0.00766134  0.00526994]]


In [6]:
'''from util import take_log
print(ssn_layer_pars_dict['logJ_2x2'][0]-take_log(ssn_layer_pars.J_2x2_m))
print(ssn_layer_pars_dict['logJ_2x2'][1]- take_log(ssn_layer_pars.J_2x2_s))
'''

"from util import take_log\nprint(ssn_layer_pars_dict['logJ_2x2'][0]-take_log(ssn_layer_pars.J_2x2_m))\nprint(ssn_layer_pars_dict['logJ_2x2'][1]- take_log(ssn_layer_pars.J_2x2_s))\n"